In [2]:
import pandas as pd
import numpy as np
import random
import math
import pickle
import json
import os
import requests
import datetime
import boto3
from botocore.exceptions import NoCredentialsError
from domino.training_sets import TrainingSetClient, model

#set DMM vars
bucket = 'wine-quality-monitoring'

#Load in data

In [3]:
print('Reading in data for batch scoring')
df = TrainingSetClient.get_training_set_version('winequality-training-domino-admin', number = 1).load_raw_pandas()

Reading in data for batch scoring


In [4]:
df.head()

id  density  volatile_acidity  chlorides  is_red  alcohol  quality
0   0   1.0010              0.27      0.045       0      8.8     5.58
1   1   0.9940              0.30      0.049       0      9.5     5.04
2   2   0.9951              0.28      0.050       0     10.1     5.34
3   3   0.9956              0.23      0.058       0      9.9     4.92
4   4   0.9956              0.23      0.058       0      9.9     5.16

In [5]:
df2 = df.append(df).reset_index(drop=True)

/tmp/ipykernel_620/2998420595.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df.append(df).reset_index(drop=True)


In [6]:
df.columns

Index(['id', 'density', 'volatile_acidity', 'chlorides', 'is_red', 'alcohol',
       'quality'],
      dtype='object')

In [7]:
##For each input feature adjust data and round/cast as necessary
#Density - 50%-150
densityJitter = df2.density.apply(lambda x : x*(random.randrange(50,150))/100).round(4)
#volatile acidity - 70%-130%
volatileAcidityJitter = df2.volatile_acidity.apply(lambda x : x*(random.randrange(70,130)/100)).round(2)
#Chlorides - 80%-120%
chloridesJitter = df2.chlorides.apply(lambda x : x*(random.randrange(80,120)/100)).round(3)
#is_red - 40%-160%
is_redJitter = df2.is_red.apply(lambda x : x*(random.randrange(40,160)/100)).round(0)
#alcohol - 90%-110%
alcoholJitter = df2.alcohol.apply(lambda x : x*(random.randrange(90,110)/100)).round(1)

#Take all the new 'jittered' variables and write to a new df
#Keep original custid and churn_Y fields
df3 = pd.DataFrame({'id': df2.id,
       'density': densityJitter, 
       'volatile_acidity': volatileAcidityJitter,
       'chlorides': chloridesJitter,
       'is_red': is_redJitter,
       'alcohol': alcoholJitter,
       'quality': df2.quality
                   })

In [8]:
df3

id  density  volatile_acidity  chlorides  is_red  alcohol  quality
0         0   1.3614              0.19      0.045     0.0      9.5     5.58
1         1   0.8250              0.37      0.044     0.0      9.2     5.04
2         2   1.2439              0.31      0.041     0.0     10.9     5.34
3         3   1.0155              0.29      0.049     0.0     10.3     4.92
4         4   0.9657              0.27      0.066     0.0      9.7     5.16
...     ...      ...               ...        ...     ...      ...      ...
12921  6491   1.1858              0.51      0.058     1.0      9.9     5.10
12922  6492   0.8855              0.49      0.074     1.0     10.4     5.60
12923  6494   1.3841              0.44      0.074     0.0     11.2     6.18
12924  6495   0.9756              0.51      0.076     1.0     10.8     5.65
12925  6496   0.8063              0.27      0.067     1.0     10.3     5.22

[12926 rows x 7 columns]

In [9]:
#Grab between 50 and 500 random rows from jittered data
df_inf = df3.sample(n = random.randint(50,100)).reset_index(drop=True)

#set up clean customer_ids
setup_ids = list(range(0, df_inf.shape[0]))
ids = list()
for i in setup_ids:
    ids.append(str(datetime.date.today())+'_'+str(setup_ids[i]))

In [10]:
df_inf['wine_id']=ids    
print('Sending {} records to model API endpoint for scoring'.format(df_inf.shape[0]))

#Set up dictionaries and lists for loops
setup_dict = {}
scoring_request = {}
results = list()

inputs = df_inf[['wine_id','density', 'volatile_acidity', 'chlorides', 'is_red', 'alcohol']]

Sending 90 records to model API endpoint for scoring


In [11]:
for n in range(inputs.shape[0]):
    for i in list(inputs.columns):
        setup_dict.update({i :list(inputs[n:n+1].to_dict().get(i).values())[0]})
        scoring_request = {'data' : setup_dict}
        
        
    response = requests.post("https://rev4mlops.domino-eval.com:443/models/646b7aa803e97e4d244dba2a/latest/model",
    auth=(
        "3OOzvWW9oCneKZnagRZ6ac4lbZ3mseyIxeOB3Yk2XvznYly3tY89UKQgDoatkm2H",
        "3OOzvWW9oCneKZnagRZ6ac4lbZ3mseyIxeOB3Yk2XvznYly3tY89UKQgDoatkm2H"
    ),
        json=scoring_request
    )
    results.append(response.json().get('result').get('prediction'))

print('Scoring complete')

Scoring complete


In [12]:
df_ground_truth=df_inf[['wine_id', 'quality']].rename({'wine_id': 'event_id', 'quality' : 'quality_GT'}, axis=1)
print(df_ground_truth.shape[0]==inputs.shape[0])
print((df_ground_truth.event_id==inputs.wine_id).sum()==df_ground_truth.shape[0])

True
True


In [13]:
gt_file_name = str('GT_Data_') + str(datetime.date.today())+str('.csv')
gt_file_path = str('/domino/datasets/local/ground_truth_data/')+gt_file_name
df_ground_truth.to_csv(gt_file_path, index=False)

In [14]:
def s3_upload(local_file, bucket):
    s3 = boto3.client('s3', aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
                      aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'])
    
    s3_file_name = '{}'.format(os.path.basename(local_file))
    
    try:
        s3.upload_file(local_file, bucket, s3_file_name)
        print(str(s3_file_name) + " Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False
    
s3_upload(gt_file_path, bucket)

print('Data Uploaded to s3 bucket at s3://{}/{}'.format(bucket, gt_file_name))
print('Done!')

GT_Data_2023-05-22.csv Upload Successful
Data Uploaded to s3 bucket at s3://wine-quality-monitoring/GT_Data_2023-05-22.csv
Done!
